## Goals
 - Take the merged predictions and evaluate the prediction accuracy using the 2 different approaches
 1. Look at the anaphora tags and then cross-reference co-reference labels
 2. Use the co-reference chains directly

In [2]:
import dill
import pandas as pd

from Settings import Settings
from collections import defaultdict
from BrattEssay import ANAPHORA

from results_common import get_essays, validate_essays, tally_essay_attributes
from process_essays_coref import get_coref_processed_essays
from metrics import get_metrics_raw

# progress bar widget
from ipywidgets import IntProgress
from IPython.display import display

DATASET = "CoralBleaching" # CoralBleaching | SkinCancer

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
stanford_coref_predictions_folder = root_folder + "CoReference/"
berkeley_coref_predictions_folder = root_folder + "CoReference/Berkeley/"
# Which algorithm?
coref_predictions_folder = stanford_coref_predictions_folder
print("CoRef Data: ", stanford_coref_predictions_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
CoRef Data:  /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/


In [3]:
training_essays = get_essays(coref_predictions_folder, "Training")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/training_processed.dill


In [4]:
test_essays = get_essays(coref_predictions_folder, "Test")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/test_processed.dill


In [5]:
all_essays = training_essays + test_essays

### Validate the Lengths

In [6]:
validate_essays(training_essays)
validate_essays(test_essays)

Essays validated
Essays validated


In [7]:
# ner_tally = tally_essay_attributes(all_essays, attribute_name="pred_ner_tags_sentences")
pos_tally = tally_essay_attributes(all_essays, attribute_name="pred_pos_tags_sentences")

## Look at the Anaphor Tags

In [8]:
from results_procesor import is_a_regular_code

cc_tally = defaultdict(int)
cr_tally = defaultdict(int)
reg_tally = defaultdict(int)
for e in all_essays:
    for sent in e.sentences:
        for wd, tags in sent:
            for t in tags:
                if is_a_regular_code(t):
                    reg_tally[t] += 1
                if ANAPHORA in t and "other" not in t:
                    if "->" in t:
                        cr_tally[t] += 1
                    elif "Anaphor:[" in t:
                        cc_tally[t] += 1

reg_tags = sorted(reg_tally.keys())
all_ana_tags = sorted(cc_tally.keys())
assert len(reg_tags) == len(all_ana_tags)
all_ana_tags

['Anaphor:[11]',
 'Anaphor:[12]',
 'Anaphor:[13]',
 'Anaphor:[14]',
 'Anaphor:[1]',
 'Anaphor:[2]',
 'Anaphor:[3]',
 'Anaphor:[4]',
 'Anaphor:[50]',
 'Anaphor:[5]',
 'Anaphor:[5b]',
 'Anaphor:[6]',
 'Anaphor:[7]']

In [9]:
NEAREST_REF_ONLY = "Nearest reference"
MAX_ANA_PHRASE = "Max ana phrase"
MAX_CHAIN_PHRASE = "Max chain phrase"
POS_ANA_FLTR = "POS ana filter"
POS_CHAIN_FLTR = "Pos chain filter"

def blank_if_none(val):
    return "-" if (val is None or not val or str(val).lower() == "none") else val

def process_sort_results(df_results):
    df_disp = df_results[["f1_score","precision","recall", 
                          NEAREST_REF_ONLY, MAX_ANA_PHRASE, MAX_CHAIN_PHRASE, POS_ANA_FLTR, POS_CHAIN_FLTR]]
    return df_disp.sort_values("f1_score", ascending=False)

def filter_test_results_to_best_training_results(df_train_raw, df_test_raw):
    # make sure sorted to the top result
    df_train_raw_sorted = df_train_raw.sort_values("f1_score", ascending=False, inplace=False)
    top_row = df_train_raw_sorted.iloc[0]
    filtered_df = df_test_raw[df_test_raw[NEAREST_REF_ONLY] == top_row[NEAREST_REF_ONLY]]
    filtered_df = filtered_df[filtered_df[MAX_ANA_PHRASE]   == top_row[MAX_ANA_PHRASE]]
    filtered_df = filtered_df[filtered_df[MAX_CHAIN_PHRASE] == top_row[MAX_CHAIN_PHRASE]]
    filtered_df = filtered_df[filtered_df[POS_ANA_FLTR]     == top_row[POS_ANA_FLTR]]
    filtered_df = filtered_df[filtered_df[POS_CHAIN_FLTR]   == top_row[POS_CHAIN_FLTR]]
    print(len(filtered_df))
    return process_sort_results(filtered_df)

## Prepare POS Tag Filters

In [10]:
pos_nouns = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "NN"])
pos_verbs = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "VB"])
pos_pronouns = {"PRP","PRP$", "WP", "WP$"}
pos_determiners = {"DT","WDT","PDT"} # the, a, which, that, etc
pos_pron_dt = pos_pronouns | pos_determiners
# for meaning of pen treebank tags - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_nouns, pos_verbs, pos_pronouns, pos_determiners, pos_pron_dt

({'NN', 'NNP', 'NNPS', 'NNS'},
 {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
 {'PRP', 'PRP$', 'WP', 'WP$'},
 {'DT', 'PDT', 'WDT'},
 {'DT', 'PDT', 'PRP', 'PRP$', 'WDT', 'WP', 'WP$'})

In [11]:
dict_pos_filter = {
            "None": None,
            "PRN": pos_pronouns,
            "DT": pos_determiners,
            "PRN+DT": pos_pron_dt
}

dict_pos_ch_filter = {
    "None": None,
    "NN": pos_nouns,
    "VB": pos_verbs,
    "NN+VB": pos_nouns | pos_verbs
}

In [12]:
phrase_len = [None,1,2,3,5,10,20]
# phrase_len = [None,1]
nearest_ref_only_values = [True,False]

# Grid Search With Anaphora Prediction Filters

In [19]:
def get_test_metrics(essays, format_ana_tags, filter_to_predicted_tags, expected_tags,
                    nearest_ref_only, pos_ana_key, pos_ch_key, max_ana_phrase_len, max_cref_phrase_len):
    
    pos_ana_filter = dict_pos_filter[pos_ana_key]
    pos_ch_filter  = dict_pos_ch_filter[pos_ch_key]
    
    proc_essays = get_coref_processed_essays(
                            essays=essays, format_ana_tags=format_ana_tags, 
                            ner_ch_filter=None, look_back_only=True,
                            filter_to_predicted_tags=filter_to_predicted_tags, 
                            max_ana_phrase_len=max_ana_phrase_len, max_cref_phrase_len=max_cref_phrase_len, 
                            pos_ana_filter=pos_ana_filter, pos_ch_filter=pos_ch_filter, 
                            nearest_ref_only=nearest_ref_only)
                        
    metrics = get_metrics_raw(proc_essays, expected_tags=expected_tags,  micro_only=True)
    row = metrics["MICRO_F1"]
    row[NEAREST_REF_ONLY] = blank_if_none(nearest_ref_only)
    row[MAX_ANA_PHRASE]   = blank_if_none(max_ana_phrase_len)
    row[MAX_CHAIN_PHRASE] = blank_if_none(max_cref_phrase_len)
    row[POS_ANA_FLTR]     = blank_if_none(pos_ana_key)
    row[POS_CHAIN_FLTR]   = blank_if_none(pos_ch_key)
    df_results = pd.DataFrame([row])
    return df_results

In [14]:
def grid_search(essays, format_ana_tags, filter_to_predicted_tags, expected_tags):

    # set up progress bar
    max_count = len(nearest_ref_only_values) * len(phrase_len) * len(phrase_len) * len(dict_pos_filter) * len(dict_pos_ch_filter)
    iprogress_bar = IntProgress(min=0, max=max_count) # instantiate the bar
    display(iprogress_bar) # display the bar

    rows_ana = []
    
    for nearest_ref_only in nearest_ref_only_values:
        for pos_ana_key, pos_ana_filter in dict_pos_filter.items():
            for pos_ch_key, pos_ch_filter in dict_pos_ch_filter.items():                
                for max_ana_phrase_len in phrase_len:
                    for max_cref_phrase_len in phrase_len:

                        proc_essays = get_coref_processed_essays(
                            essays=essays, format_ana_tags=format_ana_tags, 
                            ner_ch_filter=None, look_back_only=True,
                            filter_to_predicted_tags=filter_to_predicted_tags, 
                            max_ana_phrase_len=max_ana_phrase_len, max_cref_phrase_len=max_cref_phrase_len, 
                            pos_ana_filter=pos_ana_filter, pos_ch_filter=pos_ch_filter, 
                            nearest_ref_only=nearest_ref_only)
                        
                        metrics = get_metrics_raw(proc_essays, expected_tags=expected_tags,  micro_only=True)
                        row = metrics["MICRO_F1"]
                        row[NEAREST_REF_ONLY] = blank_if_none(nearest_ref_only)
                        row[MAX_ANA_PHRASE]   = blank_if_none(max_ana_phrase_len)
                        row[MAX_CHAIN_PHRASE] = blank_if_none(max_cref_phrase_len)
                        row[POS_ANA_FLTR]     = blank_if_none(pos_ana_key)
                        row[POS_CHAIN_FLTR]   = blank_if_none(pos_ch_key)
                        rows_ana.append(row)
                        iprogress_bar.value += 1

    df_results = pd.DataFrame(rows_ana)
    return df_results

### Training

In [15]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw = grid_search(essays=training_essays, 
                           expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=1568)

In [17]:
df_train_results = process_sort_results(df_train_raw)
df_train_results.head()

,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
14,0.038043,0.291667,0.020349,True,2,-,-,-
602,0.038043,0.291667,0.020349,True,2,-,PRN+DT,-
607,0.038043,0.291667,0.020349,True,2,10,PRN+DT,-
608,0.038043,0.291667,0.020349,True,2,20,PRN+DT,-
20,0.038043,0.291667,0.020349,True,2,20,-,-


In [20]:
# persist results
train_res_fname = coref_predictions_folder + "concept_codes_train_grid_search_ana_predictions.csv"
df_train_results.to_csv(train_res_fname, sep=",")

### Test

In [37]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw = grid_search(essays=test_essays, 
                          expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=1568)

In [40]:
process_sort_results(df_test_raw).head()

,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
915,0.05,1.0,0.025641,-,5,10,-,VB
726,0.05,1.0,0.025641,True,10,10,PRN+DT,VB
910,0.05,1.0,0.025641,-,5,-,-,VB
909,0.05,1.0,0.025641,-,3,20,-,VB
908,0.05,1.0,0.025641,-,3,10,-,VB


In [41]:
filter_test_results_to_best_training_results(df_train_raw, df_test_raw)

1


,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
14,0.047619,0.333333,0.025641,True,2,-,-,-


## Grid Search without Anaphora Predictions

### Training

In [42]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw_cref = grid_search(essays=training_essays, 
                                expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=1568)

In [43]:
process_sort_results(df_train_raw_cref).head()

,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
411,0.033333,0.031915,0.034884,True,2,10,DT,-
460,0.032448,0.032934,0.031977,True,2,10,DT,NN
558,0.032353,0.032738,0.031977,True,2,10,DT,NN+VB
412,0.032345,0.030151,0.034884,True,2,20,DT,-
406,0.032172,0.029851,0.034884,True,2,-,DT,-


### Test

In [44]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw_cref = grid_search(essays=test_essays, 
                               expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=1568)

In [45]:
process_sort_results(df_test_raw_cref).head()

,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
509,0.046512,0.250000,0.025641,True,2,10,DT,VB
516,0.045455,0.200000,0.025641,True,3,10,DT,VB
504,0.044444,0.166667,0.025641,True,2,-,DT,VB
705,0.044444,0.166667,0.025641,True,2,10,PRN+DT,VB
510,0.044444,0.166667,0.025641,True,2,20,DT,VB


In [46]:
filter_test_results_to_best_training_results(df_train_raw_cref, df_test_raw_cref)

1


,f1_score,precision,recall,Nearest reference,Max ana phrase,Max chain phrase,POS ana filter,Pos chain filter
411,0.017699,0.013514,0.025641,True,2,10,DT,-


## TODO
- ~~Make look-back-only always true~~
  - **instead, add an option to pick the nearest preceding referent only**
- ~~add a length filter to look at the length of the anaphor as well as the antecedents~~